# Case Study

In [1]:
import matplotlib.pyplot as plt

import andes
import ams

In [2]:
%matplotlib inline

In [3]:
andes.config_logger(stream_level=30)
ams.config_logger(stream_level=40)

In [4]:
!andes misc --version

Python  3.12.0
andes   1.9.3.post50+gb93e83c4
numpy   2.0.2
kvxopt  1.3.2.1
sympy   1.13.3
scipy   1.13.1
pandas  2.2.2
numba   0.60.0


In this study, we use the same OPF case cause the only difference is the `StaticGen.genfuel` and it will not impact the OPF results.

The four dynamic cases are:

``1.IL200_dyn_20wt.xlsx``, 20% wind penetration with IEEEG1

``2.IL200_dyn_20wt_ts.xlsx``, 20% wind penetration with IEEEG1 replaced by IEEEG1TS

``3.IL200_dyn_50wt.xlsx``, 50% wind penetration with IEEEG1

``4.IL200_dyn_50wt_ts.xlsx``, 50% wind penetration with IEEEG1 replaced by IEEEG1TS

In [5]:
opf_cases = ['./ACTIVSg200/IL200_opf_20wt.xlsx',
             './ACTIVSg200/IL200_opf_50wt.xlsx',
             './ACTIVSg200/IL200_opf_80wt.xlsx',]
sp1 = ams.load(opf_cases[0],
               setup=True, no_output=True,
               default_config=True)
sp2 = ams.load(opf_cases[0],
               setup=True, no_output=True,
               default_config=True)
sp3 = ams.load(opf_cases[1],
               setup=True, no_output=True,
               default_config=True)
sp4 = ams.load(opf_cases[1],
               setup=True, no_output=True,
               default_config=True)
sp5 = ams.load(opf_cases[2],
               setup=True, no_output=True,
               default_config=True)
sp6 = ams.load(opf_cases[2],
               setup=True, no_output=True,
               default_config=True)

dy_cases = ['./ACTIVSg200/1.IL200_dyn_20wt.xlsx',
            './ACTIVSg200/2.IL200_dyn_20wt_ts.xlsx',
            './ACTIVSg200/3.IL200_dyn_50wt.xlsx',
            './ACTIVSg200/4.IL200_dyn_50wt_ts.xlsx',
            './ACTIVSg200/5.IL200_dyn_80wt.xlsx',
            './ACTIVSg200/6.IL200_dyn_80wt_ts.xlsx',]

s1 = sp1.to_andes(addfile=dy_cases[0],
                  setup=False, no_output=True,
                  default_config=True)
s2 = sp2.to_andes(addfile=dy_cases[1],
                  setup=False, no_output=True,
                  default_config=True)
s3 = sp3.to_andes(addfile=dy_cases[2],
                  setup=False, no_output=True,
                  default_config=True)
s4 = sp4.to_andes(addfile=dy_cases[3],
                  setup=False, no_output=True,
                  default_config=True)
s5 = sp5.to_andes(addfile=dy_cases[4],
                  setup=False, no_output=True,
                  default_config=True)
s6 = sp6.to_andes(addfile=dy_cases[5],
                  setup=False, no_output=True,
                  default_config=True)

for ss in [s1, s2, s3, s4, s5, s6]:
    # add a Alter device to set load increase
    ss.add(model='Alter',
           param_dict=dict(t=1, model='PQ', dev='PQ_1', src='Ppf',
                           attr='v', method='+', amount=0.1))
    ss.add(model='Alter',
           param_dict=dict(t=1, model='PQ', dev='PQ_2', src='Ppf',
                           attr='v', method='+', amount=0.1))
    ss.add(model='Alter',
           param_dict=dict(t=1, model='PQ', dev='PQ_3', src='Ppf',
                           attr='v', method='+', amount=0.1))

    ss.setup()

Generating code for 1 models on 12 processes.
Generating code for 1 models on 12 processes.
Generating code for 1 models on 12 processes.
Generating code for 1 models on 12 processes.
Generating code for 1 models on 12 processes.
Generating code for 1 models on 12 processes.


In [6]:
for ss in [sp1, sp2, sp3, sp4, sp5, sp6]:
    ss.DCOPF.run(solver='CLARABEL')
    ss.DCOPF.dc2ac()

In [7]:
sp1.dyn.send(adsys=s1, routine='DCOPF')
sp2.dyn.send(adsys=s2, routine='DCOPF')
sp3.dyn.send(adsys=s3, routine='DCOPF')
sp4.dyn.send(adsys=s4, routine='DCOPF')
sp5.dyn.send(adsys=s5, routine='DCOPF')
sp6.dyn.send(adsys=s6, routine='DCOPF')

True

In [8]:
s1.PFlow.run()

True

In [9]:
_ = s1.TDS.init()

GENROU (vf range) out of typical lower limit.

   idx     | values | limit
-----------+--------+------
 GENROU_6  | 0      | 1    
 GENROU_8  | 0      | 1    
 GENROU_9  | 0      | 1    
 GENROU_10 | 0      | 1    
 GENROU_16 | 0      | 1    
 GENROU_17 | 0      | 1    
 GENROU_18 | 0      | 1    
 GENROU_20 | 0      | 1    
 GENROU_22 | 0      | 1    
 GENROU_23 | 0      | 1    
 GENROU_24 | 0      | 1    
 GENROU_25 | 0      | 1    
 GENROU_26 | 0      | 1    
 GENROU_27 | 0      | 1    
 GENROU_28 | 0      | 1    
 GENROU_29 | 0      | 1    
 GENROU_30 | 0      | 1    
 GENROU_31 | 0      | 1    
 GENROU_33 | 0      | 1    
 GENROU_34 | 0      | 1    
 GENROU_35 | 0      | 1    
 GENROU_36 | 0      | 1    
 GENROU_37 | 0      | 1    
 GENROU_38 | 0      | 1    
 GENROU_39 | 0      | 1    
 GENROU_40 | 0      | 1    
 GENROU_42 | 0      | 1    
 GENROU_43 | 0      | 1    
 GENROU_45 | 0      | 1    
 GENROU_46 | 0      | 1    
 GENROU_48 | 0      | 1    
 GENROU_49 | 0      | 1    



ValueError: bus 65.0 is not terminal of the line connecting 127.0 and 123.0. Check the data of REPCA1.bus

In [12]:
s1.Line.as_df().iloc[150:200]

,idx,u,name,bus1,bus2,Sn,fn,Vn1,Vn2,r,...,g2,trans,tap,phi,rate_a,rate_b,rate_c,owner,xcoord,ycoord
uid,,,,,,,,,,,,,,,,,,,,,
150,Line_151,1.0,Line 100-179,100,179,100.0,60.0,115.0,115.0,0.007093,...,0.0,0.0,1.0,0.0,2.2110,0.0,0.0,None,None,None
151,Line_152,1.0,Line 100-184,100,184,100.0,60.0,115.0,115.0,0.012745,...,0.0,0.0,1.0,0.0,2.2110,0.0,0.0,None,None,None
152,Line_153,1.0,Line 110-101,110,101,100.0,60.0,115.0,115.0,0.001960,...,0.0,0.0,1.0,0.0,2.2110,0.0,0.0,None,None,None
153,Line_154,1.0,Line 101-117,101,117,100.0,60.0,115.0,115.0,0.005245,...,0.0,0.0,1.0,0.0,2.2110,0.0,0.0,None,None,None
154,Line_155,1.0,Line 101-141,101,141,100.0,60.0,115.0,115.0,0.003144,...,0.0,0.0,1.0,0.0,2.2110,0.0,0.0,None,None,None
155,Line_156,1.0,Line 103-102,103,102,100.0,60.0,115.0,230.0,0.001709,...,0.0,0.0,1.0,0.0,2.5000,0.0,0.0,None,None,None
156,Line_157,1.0,Line 104-102,104,102,100.0,60.0,13.8,230.0,0.002100,...,0.0,0.0,1.0,0.0,1.3000,0.0,0.0,None,None,None
157,Line_158,1.0,Line 105-102,105,102,100.0,60.0,13.8,230.0,0.000970,...,0.0,0.0,1.0,0.0,2.6000,0.0,0.0,None,None,None
158,Line_159,1.0,Line 102-128,102,128,100.0,60.0,230.0,230.0,0.002700,...,0.0,0.0,1.0,0.0,4.0236,0.0,0.0,None,None,None


In [ ]:
for ss in [s1, s2, s3, s4, s5, s6]:
    # Constant load
    ss.PQ.config.p2p = 1
    ss.PQ.config.q2q = 1
    ss.PQ.config.p2z = 0
    ss.PQ.config.q2z = 0

    ss.PFlow.run()

    _ = ss.TDS.init()

In [ ]:
for ss in [s1, s2, s3, s4, s5, s6]:

    ss.TDS.config.tf = 80
    ss.TDS.config.criteria = 0
    ss.TDS.config.no_tqdm = True

    ss.TDS.run()

In [ ]:
legends = [r'Case1: IEEEG1 20\% Wind',
           r'Case2: IEEEG1TS 20\% Wind',
           r'Case3: IEEEG1 50\% Wind',
           r'Case4: IEEEG1TS 50\% Wind']
colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red']
linestyles = ['-', '--', '-.', ':']

In [ ]:
plt.style.use('default')
fig0, ax0 = plt.subplots(1, 2, figsize=(12, 6), dpi=150)
plt.subplots_adjust(wspace=0.15)

a = (0,)
wmin, wmax = 86.48, 86.52
_ = s1.TDS.plt.plot(s1.REGCA1.Pe,
                    a=a,
                    ymin=wmin, ymax=wmax,
                    ytimes=s1.config.mva,
                    yheader=[legends[0]],
                    show=False, grid=True,
                    ylabel='Wind Generation [MW]',
                    colors=[colors[0]],
                    linestyles=[linestyles[0]],
                    fig=fig0, ax=ax0[0],)
_ = s2.TDS.plt.plot(s2.REGCA1.Pe,
                    a=a,
                    ytimes=s2.config.mva,
                    yheader=[legends[1]],
                    show=False,
                    colors=[colors[1]],
                    linestyles=[linestyles[1]],
                    fig=fig0, ax=ax0[0],)
_ = s3.TDS.plt.plot(s3.REGCA1.Pe,
                    a=a,
                    ytimes=s3.config.mva,
                    yheader=[legends[2]],
                    show=False,
                    colors=[colors[2]],
                    linestyles=[linestyles[2]],
                    fig=fig0, ax=ax0[0],)
_ = s4.TDS.plt.plot(s4.REGCA1.Pe,
                    a=a,
                    ytimes=s4.config.mva,
                    yheader=[legends[3]],
                    show=False,
                    colors=[colors[3]],
                    linestyles=[linestyles[3]],
                    fig=fig0, ax=ax0[0],)

smin, smax = 8, 11
a=(10,)
_ = s1.TDS.plt.plot(s1.GENROU.Pe,
                    a=a,
                    ymin=smin, ymax=smax,
                    ytimes=s1.config.mva,
                    yheader=[legends[0]],
                    show=False, grid=True,
                    ylabel='GENROU Generation [MW]',
                    colors=[colors[0]],
                    linestyles=[linestyles[0]],
                    fig=fig0, ax=ax0[1],)
_ = s2.TDS.plt.plot(s2.GENROU.Pe,
                    a=a,
                    ytimes=s2.config.mva,
                    yheader=[legends[1]],
                    show=False,
                    colors=[colors[1]],
                    linestyles=[linestyles[1]],
                    fig=fig0, ax=ax0[1],)
_ = s3.TDS.plt.plot(s3.GENROU.Pe,
                    a=a,
                    ytimes=s3.config.mva,
                    yheader=[legends[2]],
                    show=False,
                    colors=[colors[2]],
                    linestyles=[linestyles[2]],
                    fig=fig0, ax=ax0[1],)
_ = s4.TDS.plt.plot(s4.GENROU.Pe,
                    a=a,
                    ytimes=s4.config.mva,
                    yheader=[legends[3]],
                    show=False,
                    colors=[colors[3]],
                    linestyles=[linestyles[3]],
                    fig=fig0, ax=ax0[1],)

In [ ]:
plt.style.use('default')
fig, ax = plt.subplots(1, 2, figsize=(12, 6), dpi=150)
plt.subplots_adjust(wspace=0.15)

fmin, fmax = 59.9, 60.01
a = (46,)
_ = s1.TDS.plt.plot(s1.GENROU.omega,
                    a=a,
                    ymin=fmin, ymax=fmax,
                    ytimes=s1.config.freq,
                    show=False, grid=True,
                    ylabel='Slack Gen. Freq. [Hz]',
                    yheader=[legends[0]],
                    colors=[colors[0]],
                    linestyles=[linestyles[0]],
                    fig=fig, ax=ax[0])
_ = s2.TDS.plt.plot(s2.GENROU.omega,
                    a=a,
                    ytimes=s2.config.freq,
                    show=False,
                    yheader=[legends[1]],
                    colors=[colors[1]],
                    linestyles=[linestyles[1]],
                    fig=fig, ax=ax[0],)
_ = s3.TDS.plt.plot(s3.GENROU.omega,
                    a=a,
                    ytimes=s3.config.freq,
                    show=False,
                    yheader=[legends[2]],
                    colors=[colors[2]],
                    linestyles=[linestyles[2]],
                    fig=fig, ax=ax[0],)
_ = s4.TDS.plt.plot(s4.GENROU.omega,
                    a=a,
                    ytimes=s4.config.freq,
                    show=False,
                    yheader=[legends[3]],
                    colors=[colors[3]],
                    linestyles=[linestyles[3]],
                    fig=fig, ax=ax[0],)

a = (0,)
tmin, tmax = 1.1, 4.5
_ = s1.TDS.plt.plot(s1.IEEEG1.pout,
                    a=a,
                    ytimes=s1.config.mva,
                    ymin=tmin, ymax=tmax,
                    show=False, grid=True,
                    ylabel='TurbineGov Power [MW]',
                    yheader=[legends[0]],
                    colors=[colors[0]],
                    linestyles=[linestyles[0]],
                    fig=fig, ax=ax[1],)
_ = s2.TDS.plt.plot(s2.IEEEG1TS2.pout,
                    a=a,
                    ytimes=s2.config.mva,
                    show=False,
                    yheader=[legends[1]],
                    colors=[colors[1]],
                    linestyles=[linestyles[1]],
                    fig=fig, ax=ax[1],)
_ = s3.TDS.plt.plot(s3.IEEEG1.pout,
                    a=a,
                    ytimes=s2.config.mva,
                    show=False,
                    yheader=[legends[2]],
                    colors=[colors[2]],
                    linestyles=[linestyles[2]],
                    fig=fig, ax=ax[1],)
_ = s4.TDS.plt.plot(s4.IEEEG1TS2.pout,
                    a=a,
                    ytimes=s2.config.mva,
                    show=False,
                    yheader=[legends[3]],
                    colors=[colors[3]],
                    linestyles=[linestyles[3]],
                    fig=fig, ax=ax[1],)

In [ ]:
fig1, ax1 = plt.subplots(1, 2, figsize=(12, 6), dpi=150)
plt.subplots_adjust(wspace=0.25)

a = (0,)
_ = s2.TDS.plt.plot(s2.IEEEG1TS2.TS.y,
                    a=a,
                    ycalc=lambda x: 1 - x,
                    show=False, grid=True,
                    ylabel='Thermal Storage Gain',
                    yheader=[legends[1]],
                    color=colors[1],
                    linestyles=linestyles[1],
                    fig=fig1, ax=ax1[0])
_ = s4.TDS.plt.plot(s4.IEEEG1TS2.TS.y,
                    a=a,
                    ycalc=lambda x: 1 - x,
                    show=False,
                    yheader=[legends[3]],
                    color=colors[3],
                    linestyles=linestyles[3],
                    fig=fig1, ax=ax1[0])

a = (0,)
_ = s1.TDS.plt.plot(s1.IEEEG1.GV,
                    a=a,
                    show=False, grid=True,
                    ymin=0.01, ymax=0.045,
                    ylabel='Valve Position',
                    yheader=[legends[0]],
                    color=colors[0],
                    linestyles=linestyles[0],
                    fig=fig1, ax=ax1[1])
_ = s2.TDS.plt.plot(s2.IEEEG1TS2.GV,
                    a=a,
                    show=False,
                    yheader=[legends[1]],
                    color=colors[1],
                    linestyles=linestyles[1],
                    fig=fig1, ax=ax1[1])
_ = s3.TDS.plt.plot(s3.IEEEG1.GV,
                    a=a,
                    show=False,
                    yheader=[legends[2]],
                    color=colors[2],
                    linestyles=linestyles[2],
                    fig=fig1, ax=ax1[1])
_ = s4.TDS.plt.plot(s4.IEEEG1TS2.GV,
                    a=a,
                    show=False,
                    yheader=[legends[3]],
                    color=colors[3],
                    linestyles=linestyles[3],
                    fig=fig1, ax=ax1[1])